### Contador de palabras.

Este código esta diseñado para realizar el conteo de determinadas palabras en un PDF aplicando un OCR, en concreto tesseract.

El listado de palabras se inserta con un archivo csv, txt o excel. Más abajo hay más información al respecto.

Para un uso habitual del código, hay que subir ambos archivos, PDF y archivo de palabras, a la carpeta de Colab, situada a la izquierda. Posteriormente se cambian los parámetros pertinentes en la última celda y en la pestaña de "entorno de ejecución" se da a "ejecutar todas"

El código ha sido creado por José Ignacio Salas Cáceres. Correo de contacto: jose.salas@ulpgc.es

In [1]:
# Instalamos Los paquetes necesarios.
!sudo apt install tesseract-ocr
!pip install pytesseract
!apt-get install poppler-utils
!pip install pdf2image
!apt-get install -y libpoppler-cpp-dev
!pip install --upgrade pdf2image
!pip install python-Levenshtein
!pip install pypdf
!pip install fpdf
! wget https://raw.githubusercontent.com/tesseract-ocr/tessdata_best/master/spa_old.traineddata #https://stackoverflow.com/questions/57968216/google-colab-how-do-i-install-traineddata-file-for-pytesseract
! sudo mv "/content/spa_old.traineddata" "/usr/share/tesseract-ocr/4.00/tessdata"
! wget https://raw.githubusercontent.com/tesseract-ocr/tessdata_best/master/spa.traineddata #https://stackoverflow.com/questions/57968216/google-colab-how-do-i-install-traineddata-file-for-pytesseract
! sudo mv "/content/spa.traineddata" "/usr/share/tesseract-ocr/4.00/tessdata"

# Descargamos los scripts con las funciones necesarias de github https://github.com/jisalascaceres/Apoyo-Filologia
! wget https://raw.githubusercontent.com/jisalascaceres/Apoyo-Filologia/main/Contador_Palabras/Word_count.py
! wget https://raw.githubusercontent.com/jisalascaceres/Apoyo-Filologia/main/Contador_Palabras/Utils.py

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (3,663 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [7]:
import pandas as pd
import numpy as np
import sys
import random
import pypdf
from tqdm import tqdm
import os
import time
import Levenshtein
from nltk.tokenize import word_tokenize
import pytesseract
from pdf2image import convert_from_path
from pypdf import PdfReader
import nltk
nltk.download('punkt_tab')
from fpdf import FPDF

from Utils import *
from Word_count import *

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


### Parametros:
- **name_excel**: Nombre del archivo de entrada con las palabras a buscar. Puede ser .csv, .txt o excel. Más abajo hay más información sobre el formato.
- **name_pdf**: Nombre del archivo pdf del que se va a realizar el conteo de palabras.
- **Página inicial**: Página del PDF desde la que se quiere empezar a contar.
- **Página final**: Página del PDF en la que se quiere terminar de contar. Si se pone -1, automáticamente coge la última página del PDF.
- **Distancia**: Distancia máxima entre dos palabras para guardar coincidencias no exactas. La distancia se calcula como el número de ediciones que hay que realizar a la palabra para que se convierta en otra. Para más información, busca "distancia Levenshtein".
- **Nombre archivo resultante**: Nombre del archivo csv que se creará con los resultados.
- **Formato**: Formato del archivo resultante. (.xlsx, .csv o .txt)
- **Aplicar_OCR** = Si hace falta o no aplicar OCR al PDF. Si el archivo esta digitalizado ya, no hará falta hacerlo.
- **Guardar_Text** = En caso de aplicar OCR, si quieres guardar el texto digitalizado por el OCR en otro PDF. Este nuevo PDF respetará la página en la que se ha encuentra cada palabra, pero no el formato exacto. Será texto plano.
- **Buscar_substrings** = Si quieres buscar la palabra dentro de otras palabras detectadas. Esto aporta robustez a no detectar un espacio. O puede detectar palabras compuestas.
- **Aplicar_Conteo** = Si se quiere contar las palabras o solo realizar el OCR.


Formato de archivo entrada:
 - En el caso de un .csv, el archivo debe tener una sola columna con palabras a buscar separadas por espacios.
 - En el caso de un excel (.xlsx), el archivo debe tener una sola columna con las palabras en la primera página.
 - En el caso de un .txt, el archivo debe contener todas las palabras separadas por comas, puntos o la letra enter (retorno de carro).

In [144]:
name_excel = 'Hispanismos.xlsx'
name_pdf = 'hola.pdf'
nombre_archivo_resultante = 'Output.xlsx'

path_csv = '/content/' + name_excel
path_pdf = '/content/' + name_pdf

Pagina_inicial = 1
Pagina_final = -1
Distancia = 2

Aplicar_OCR = False
Guardar_Texto = True
Buscar_substrings = False
Aplicar_Conteo = True
formato = '.xlsx'



Perform_word_count(path_pdf, path_csv, Pagina_inicial, Pagina_final, Distancia, nombre_archivo_resultante, formato
                   ,'spa',Aplicar_OCR=Aplicar_OCR,save_text=Guardar_Texto,Aplicar_Conteo = Aplicar_Conteo,Buscar_substrings=Buscar_substrings)


Iniciando análisis de texto...
PDF:  /content/hola.pdf
Archivo de entrada:  /content/Hispanismos.xlsx
La página final debe ser menor que el número de páginas del pdf, que es 2, se establece el parámetro a este último.
Número de páginas del PDF:  2
Leyendo desde la página 1 hasta la página 2
Leyendo el PDF sin aplicar OCR.


100%|██████████| 2/2 [00:00<00:00, 22.61it/s]


Guardando el texto digitalizado...
PDF saved as: output.pdf
Buscando palabras...
Iterando por la lista de palabras. Buscandolas en el texto. Esto puede tardar.


10it [00:00, 599.36it/s]

Guardando resultados...
Archivo guardado en:  Output.xlsx
